# Sistemas Inteligentes

## Practica 1: Búsqueda en espacio de estados

### Declaracion e implementacion de clases necesarias

In [ ]:

class Estado:
    def __init__(self, valor):
          self.valor = valor

     def __eq__(self, other):
        return isinstance(other, Estado) and self.valor == other.valor

     def __hash__(self):
        return hash(self.valor)

class Accion:
    def __init__(self, descripcion):
        self.descripcion = descripcion

class Nodo:
    def __init__(self, estado, padre=None, accion=None):
        self.estado = estado
        self.padre = padre
        self.accion = accion

    def hijo(self, accion):
        nuevo_estado = self.estado.resultado(accion)
        return Nodo(nuevo_estado, self, accion)

    def solucion(self):
        acciones = []
        nodo = self
        while nodo.padre is not None:
            acciones.append(nodo.accion)
            nodo = nodo.padre
        acciones.reverse()
        return acciones

class Problema:
    def __init__(self, estado_inicial, estado_objetivo):
        self.estado_inicial = estado_inicial
        self.estado_objetivo = estado_objetivo

    def es_objetivo(self, estado):
        return estado == self.estado_objetivo

    def acciones(self, estado):
        raise NotImplementedError

    def resultado(self, estado, accion):
        raise NotImplementedError
    

### 1: Estrategia de búsqueda no informada

#### 1.1: Búsqueda primero en anchura

In [ ]:
def busqueda_primero_en_anchura(problema):
    # Inicializar la frontera con el nodo inicial
    nodo_inicial = Nodo(problema.estado_inicial)
    if problema.es_objetivo(nodo_inicial.estado):
        return nodo_inicial.solucion()
    
    frontera = [nodo_inicial]
    explorados = set()

    while frontera:
        nodo = frontera.pop(0)
        explorados.add(nodo.estado)

        for accion in problema.acciones(nodo.estado):
            hijo = nodo.hijo(accion)
            if hijo.estado not in explorados and hijo not in frontera:
                if problema.es_objetivo(hijo.estado):
                    return hijo.solucion()
                frontera.append(hijo)
    
    return None
